# Quickstart

Following [this tutotial](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)

## Working with data

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch offers domain-specific libraries such as `TorchText`, `TorchVision`, and `TorchAudio`, all of which include datasets. For this tutorial, we will be using a `TorchVision` dataset.

The `torchvision.datasets` module contains Dataset objects for many real-world vision data like CIFAR, COCO (full list here). In this tutorial, we use the `FashionMNIST` dataset. Every TorchVision Dataset includes two arguments: transform and target_transform to modify the samples and labels respectively.

In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



We pass the Dataset as an argument to `DataLoader`. This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating models

To define a neural network in PyTorch, we create a class that inherits from `nn.Module`. We define the layers of the network in the `__init__` function and specify how data will pass through the network in the forward function. To accelerate operations in the neural network, we move it to the GPU if available.

In [34]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


### nn.Flaten
`torch.nn.Flatten(start_dim=1, end_dim=- 1)`

Flattens a contiguous range of dims into a tensor. For use with Sequential.

Parameters:
- start_dim (int) – first dim to flatten (default = 1).
- end_dim (int) – last dim to flatten (default = -1).


In [13]:
input = torch.randn(32, 1, 5, 5)
input.shape

torch.Size([32, 1, 5, 5])

In [17]:
m = nn.Flatten()
output = m(input)
print(output.size())

torch.Size([32, 25])


In [18]:
m = nn.Flatten(0, 2)
output = m(input)
output.size()

torch.Size([160, 5])

In [6]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Optimizing the Model Parameters
To train a model, we need a loss function and an optimizer.

In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [20]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model’s performance against the test dataset to ensure it is learning.

In [21]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (epochs). During each epoch, the model learns parameters to make better predictions. We print the model’s accuracy and loss at each epoch; we’d like to see the accuracy increase and the loss decrease with every epoch.

In [24]:
%%time
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.296764  [    0/60000]
loss: 2.288813  [ 6400/60000]
loss: 2.265554  [12800/60000]
loss: 2.266268  [19200/60000]
loss: 2.244622  [25600/60000]
loss: 2.215903  [32000/60000]
loss: 2.232400  [38400/60000]
loss: 2.187399  [44800/60000]
loss: 2.193706  [51200/60000]
loss: 2.170219  [57600/60000]
Test Error: 
 Accuracy: 48.1%, Avg loss: 2.150659 

Epoch 2
-------------------------------
loss: 2.157178  [    0/60000]
loss: 2.149856  [ 6400/60000]
loss: 2.086166  [12800/60000]
loss: 2.108117  [19200/60000]
loss: 2.055225  [25600/60000]
loss: 1.997427  [32000/60000]
loss: 2.040715  [38400/60000]
loss: 1.942988  [44800/60000]
loss: 1.957711  [51200/60000]
loss: 1.910192  [57600/60000]
Test Error: 
 Accuracy: 53.0%, Avg loss: 1.881658 

Epoch 3
-------------------------------
loss: 1.910236  [    0/60000]
loss: 1.886373  [ 6400/60000]
loss: 1.758327  [12800/60000]
loss: 1.807700  [19200/60000]
loss: 1.704001  [25600/60000]
loss: 1.652482  [32000/600

## Saving Models

In [23]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
